In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp

import utilityfunctions as uf

## Review

The steps to calculate PCA are:
* (If appropriate) normalize the variables to be in the range 0-1
* Center the data
* Compute the covariance matrix
* Compute the eigenvectors and eigenvalues; the eigenvectors tell us the direction of variance, and the eigenvalues tell us the amount of variance
* Get an ordering over the eigenvalues
* Sort the eigenvalues and eigenvectors accordingly
* Compute the proportional variance (how much bigger?) accounted for by each principal component
* Compute the cumulative sum of the proportional variance (tells us how many eigenvectors we need to explain a desired amount of variance)
* Examine the principal components. Select v' of them.
* Project the data into PCA space
* Reconstruct the data

## Today

Today, we are going to look at what happens when we preprocess the data in different ways before fitting a PCA.

So we will look again at the summary statistics for the data.

We will preprocess the data in four ways:
* center only
* minmax global
* minmax local
* zscore

In [ ]:
# Things we need from Day 19

# Let's split off the labels
def split(data, ycol):
    y = data[:, ycol]
    xfirst = data[:, 0:ycol]
    xsecond = data[:, ycol+1:data.shape[1]]
    return (np.hstack((xfirst, xsecond)), y)

# center
def center(data):
    centered = data - np.mean(data, axis=0)
    return centered

# preprocess
def preprocess(data, minmax=False, local=False, zscore=False):
    if minmax == True and zscore == True:
        print("Nope, won't do that!")
        return data
    elif minmax == True:
        if local == False:
            data = uf.minmaxGlobal(data)
        else:
            data = uf.minmaxLocal(data)
        return center(data)
    elif zscore == True:
        return uf.zScore(data)   

In [ ]:
# This is most of the code from Day 19 in one function; it fits a PCA and prints out all kinds of things along the way
def pca_with_plots(data):
    # covariance
    covariance_matrix = (data.T @ data) / (data.shape[0] - 1)
    print("covariance matrix")
    print(covariance_matrix.shape)

    # Let's look at the covariance matrix
    fig = plt.figure(figsize=(12,12))
    sns.heatmap(pd.DataFrame(covariance_matrix), xticklabels=columns, yticklabels=columns, annot=False, cmap='PuOr')
    plt.show()

    # svd
    (evals, evectors) = np.linalg.eig(covariance_matrix)

    # sort
    evals_order = np.argsort(evals)[::-1]
    evals_sorted = evals[evals_order]
    evectors_sorted = evectors[:, evals_order]

    # proportional variance
    evals_sum = np.sum(evals_sorted)
    proportional_vars = [e / evals_sum for e in evals_sorted]

    # cumulative sum of proportional variance
    cumulative_sum = np.cumsum(proportional_vars)

    # Let's look at the proportional variance
    plt.figure(figsize=(6, 4))
    plt.bar(range(len(proportional_vars)), proportional_vars, alpha=0.5, align='center',
            label='Proportional variance')
    plt.ylabel('Proportional variance ratio')
    plt.xlabel('Ranked Principal Components')
    plt.title("Scree Graph")
    plt.legend(loc='best')
    plt.tight_layout()

    fig = plt.figure(figsize=(6,4))
    ax1 = fig.add_subplot(111)
    ax1.plot(cumulative_sum)
    ax1.set_ylim([0,1.0])
    ax1.set_xlabel('Number of Principal Components')
    ax1.set_ylabel('Cumulative explained variance')
    ax1.set_title('Elbow Plot')
    plt.show()
    
    return evals_sorted, evectors_sorted

In [ ]:
columns=['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','chorus_hit','sections','popularity']
data = np.array(np.genfromtxt('data/spotify_dataset.csv', delimiter=',', skip_header=1, usecols=(2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17), dtype=float, encoding='utf-8'))  

print(data.shape)
print(uf.getSummaryStatistics(data))

In [ ]:
centered_data = center(data)
print("centered data")
print(centered_data.shape)
print(pd.DataFrame(uf.getSummaryStatistics(centered_data)))

evals_sorted, evectors_sorted = pca_with_plots(centered_data)

In [ ]:
minmaxed_data = preprocess(data, minmax=True, local=False, zscore=False)
print("minmaxed data")
print(minmaxed_data.shape)
print(pd.DataFrame(uf.getSummaryStatistics(minmaxed_data)))

evals_sorted, evectors_sorted = pca_with_plots(minmaxed_data)

In [ ]:
minmaxed_data = preprocess(data, minmax=True, local=True, zscore=False)
print("minmaxed data")
print(minmaxed_data.shape)
print(pd.DataFrame(uf.getSummaryStatistics(minmaxed_data)))

evals_sorted, evectors_sorted = pca_with_plots(minmaxed_data)

In [ ]:
zscore_data = preprocess(data, minmax=False, zscore=True)
print("zscored data")
print(zscore_data.shape)
print(pd.DataFrame(uf.getSummaryStatistics(zscore_data)))

evals_sorted, evectors_sorted = pca_with_plots(zscore_data)

Resources:
* https://plotly.com/python/pca-visualization/
* https://wendynavarrete.com/principal-component-analysis-with-numpy/
* https://dev.to/akaame/implementing-simple-pca-using-numpy-3k0a